In [13]:
import os
import numpy as np
import pandas as pd
import slingpy as sp
from genedisco.datasets.features.hgnc_names import HGNCNames
from slingpy.data_access.data_sources.hdf5_tools import HDF5Tools
from slingpy.data_access.data_sources.hdf5_data_source import HDF5DataSource
from slingpy.data_access.data_sources.abstract_data_source import AbstractDataSource

import matplotlib.pyplot as plt
from screen import ScreenData
import scanpy as sc
%matplotlib inline
import matplotlib

In [14]:
dfs = {}

In [16]:
dfs['Adenosine'] = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name='Adenosine')

dfs['TGFb'] = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name='TGFb')

dfs['Cyclosporine'] = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name='Cyclosporine')

dfs['Tacrolimus'] = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name='Tacrolimus')

In [17]:
dfs['Adenosine']

,id,num,neg|score,neg|p-value,neg|fdr,neg|rank,neg|goodsgrna,neg|lfc,pos|score,pos|p-value,pos|fdr,pos|rank,pos|goodsgrna,pos|lfc
0,NKX2-6,8,9.991200e-01,9.991200e-01,0.999842,19347,0,0.54823,7.518200e-07,0.000007,0.138614,1,7,0.54823
1,ICA1,6,9.902100e-01,9.902600e-01,0.999312,19189,0,0.72120,2.352400e-05,0.000122,0.837534,2,5,0.72120
2,SF3B3,8,4.530300e-02,1.476800e-01,0.991027,2838,4,-0.18787,2.619100e-05,0.000200,0.837534,3,1,-0.18787
3,MTHFD1,8,9.562300e-01,9.575100e-01,0.997828,18558,1,0.47062,3.206900e-05,0.000243,0.837534,4,7,0.47062
4,RNF2,8,8.097900e-01,8.952800e-01,0.997828,17233,1,0.48480,3.762900e-05,0.000276,0.837534,5,7,0.48480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19359,REM1,8,1.635100e-04,1.121600e-03,0.812516,25,8,-0.25078,9.998400e-01,0.999840,0.999961,19360,0,-0.25078
19360,RDM1,8,9.493200e-05,6.700700e-04,0.654703,17,8,-0.48221,9.999100e-01,0.999900,0.999961,19361,0,-0.48221
19361,C10orf53,8,8.415300e-05,5.974600e-04,0.642739,16,8,-0.35762,9.999200e-01,0.999910,0.999961,19362,0,-0.35762
19362,EAF2,8,8.148400e-05,5.780300e-04,0.642739,15,8,-0.36600,9.999200e-01,0.999910,0.999961,19363,0,-0.36600


In [7]:
save_directory = '/dfs/user/yhr/genedisco/genedisco/datasets/screens/'

### Genedisco/Slingpy dataloader

In [10]:
screen_name = 'TGFb'
h5_file = os.path.join(save_directory, "carnevale_2022_{}.h5".format(screen_name))
if not os.path.exists(h5_file):
    df = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name=screen_name)
    df = df.set_index('id')

    df.index = df.index.astype('str')
    rows_to_drop = [v for v in df.index if 'CTRL' in v]
    rows_to_drop = [v for v in df.index if v[:4]=='2021']
    df = df.drop(rows_to_drop)

    gene_names = df.index.values.tolist()
    name_converter = HGNCNames(save_directory)
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    gene_names, data = df.index.values.tolist(), df[['pos|lfc']].values.astype(np.float32)

    
    """
    HDF5Tools.save_h5_file(h5_file,
                           data,
                           "carnevale_2022_{}.h5".format(screen_name),
                           column_names=["log-fold-change"],
                           row_names=gene_names)
    """
#data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

/dfs/user/yhr/causal/lib/python3.8/site-packages/genedisco/datasets/features/hgnc_names.py:95: DtypeWarning: Columns (31,38) have mixed types.Specify dtype option on import or set low_memory=False.
  previous_mapping = self.get_hgnc_mapping(to_id="symbol", from_id=other_id_name)


In [11]:
data

array([[ 0.14561 ],
       [ 0.098093],
       [-0.065836],
       ...,
       [ 0.29436 ],
       [-0.11371 ],
       [ 0.15471 ]], dtype=float32)

In [ ]:
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

### AI RA dataloader

In [7]:
for screen_name in ['Adenosine', 'TGFb', 'Cyclosporine', 'Tacrolimus']:
    out_file = os.path.join("/dfs/user/yhr/research_assistant/datasets/ground_truth_Carnevale_2022_{}.csv".format(screen_name))
    df = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name=screen_name)
    df = df.set_index('id')

    df.index = df.index.astype('str')
    rows_to_drop = [v for v in df.index if 'CTRL' in v]
    rows_to_drop = [v for v in df.index if v[:4]=='2021']
    df = df.drop(rows_to_drop)

    gene_names = df.index.values.tolist()
    name_converter = HGNCNames('/dfs/user/yhr/genedisco/genedisco/datasets/screens/')
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    df['pos|lfc'] = df['pos|lfc'].values.astype(np.float32)
    df = df.loc[:,['pos|lfc']]
    df = df.rename(columns = {'pos|lfc':'log-fold-change'})
    df = df.reset_index().rename(columns={'index':'Gene'})

    df.to_csv(out_file, index=0)

/dfs/user/yhr/causal/lib/python3.8/site-packages/genedisco/datasets/features/hgnc_names.py:95: DtypeWarning: Columns (31,38) have mixed types.Specify dtype option on import or set low_memory=False.
  previous_mapping = self.get_hgnc_mapping(to_id="symbol", from_id=other_id_name)


In [3]:
for screen_name in ['Adenosine', 'TGFb', 'Cyclosporine', 'Tacrolimus']:
    file_path = os.path.join("/dfs/user/yhr/research_assistant/datasets/ground_truth_Carnevale_2022_{}.csv".format(screen_name))
    id_col = 'Gene'
    val_col = 'log-fold-change'
    bio_taskname = 'Carnevale22_{}'.format(screen_name)
    
    screendata = ScreenData(file_path=file_path, id_col=id_col,
           val_col=val_col, bio_taskname=bio_taskname, save=True)
    screendata.identify_hits(type_='gaussian', save=True)